<a href="https://colab.research.google.com/github/azeghost/LMDB_Datasets/blob/master/LMDB_Write_and_Read_R01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Clonning Repo***

In [149]:
!mkdir ./temp
!git clone 'https://github.com/azeghost/LMDB_Datasets.git'

mkdir: cannot create directory ‘./temp’: File exists
Cloning into 'LMDB_Datasets'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5 (delta 0), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.


**Installing Caffe Lirary (CPU or GPU)**

In [0]:
#Caffe Installation:

#CPU
!apt install caffe-cpu

#GPU
!apt install caffe-gpu

**Required Packages Importing**

In [0]:
#Packages Importing:
import numpy as np
import lmdb
import caffe

**From Numpy Array to LMDB**

In [0]:
#Write Function:
def writeArray(file_name, Data, Label, index):
  """Creates LMDB Folder from for a given folder.

  # Arguments
      file_name: string name for LMDB folder to be created. 
      Data: data to be converted in Numpy Array format.
      Labels: label for the data in Numpy Array format.
      index: the index to be assigned for the data in LMDB file.

  # Returns
      True: if LMDB folder was created
      None: otherwise
  """

  X = Data.copy()
  y = Label.copy()

  map_size = X.nbytes * 10
  channels = X.shape[0]
  height = X.shape[1]
  width =  X.shape[2]
  idx = index

  env = lmdb.open(file_name, map_size=map_size)

  with env.begin(write=True) as txn:
    # txn is a Transaction object
    datum = caffe.proto.caffe_pb2.Datum()
    datum.channels = channels
    datum.height = height
    datum.width = width
    datum.data = X.tobytes()  # or .tostring() if numpy < 1.9
    datum.label = int(y)

    str_id = '{:08}'.format(idx)
    print(str_id)

    # The encode is only essential in Python 3
    txn.put(str_id.encode('ascii'), datum.SerializeToString())

    return True

**From LMDB to Numpy Array**

In [0]:
#Read Function:
def readArray(file_name, index_range = (0,0)):
  """Reads LMDB files from in a given folder and converts them to Numpy Arrays.

  # Arguments
      file_name: string name for LMDB folder contains LMDB files to be converted t Numpy Arrays format.
      index_range: tuple contains range of data to be parsed to Numpy Array in LMDB file.

  # Returns
      x: Numpy Array for the resulted data.
      y: Numpy Array for the resulted labels.
  """

  cursor_len = 0
  x, y = [], []
  lower = index_range[0]
  upper = index_range[1]

  env = lmdb.open(file_name, readonly=True)
  with env.begin() as txn:
    cursor = txn.cursor()
    cursor_len = len(list(cursor))

    for i in range(lower, upper+1):
      idx = '{:08}'.format(i).encode('ascii')
      raw_datum = txn.get(idx)
      datum = caffe.proto.caffe_pb2.Datum()
      datum.ParseFromString(raw_datum)

      flat_x = np.fromstring(datum.data, dtype=np.uint8)
      x.append(flat_x.reshape(datum.channels, datum.height, datum.width))
      y.append(datum.label)

  x, y = np.asarray(x), np.asarray(y)

  return x, y

**Return LMDB File keys**

In [0]:
#LMDB keys function
def lmdbKeys(file_name):
  """Reads LMDB file keys.

  # Arguments
      file_name: string name for LMDB folder contains LMDB files to return back the keys from.

  # Returns
      keys: list of LMDB file keys.
  """
  env = lmdb.open(file_name, readonly=True)
  keys = []
  with env.begin() as txn:
    cursor = txn.cursor()
    print(crusor)
    for key, value in cursor:
      keys.append(key)

  return keys

*Below loop for looping and reading the images, to be modified to match the "create_image_lists" function*

In [82]:
import cv2
import glob

im = glob.glob('/content/Pokemon/*.png')
file_name = 'TEST_LMDB'
for idx, img in enumerate(im):
  i = cv2.imread(img, -1)
  i = np.asarray(i)
  print(i.shape)
  writeArray(file_name, i, np.asarray([idx]), idx)

(400, 400, 4)
00000000
(400, 400, 4)
00000001
(400, 400, 4)
00000002
